<a href="https://colab.research.google.com/github/alik604/Notebooks/blob/master/amonalyDetection/Intrusion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection 

### Dataset from https://www.kaggle.com/what0919/intrusion-detection

### Sample code used: https://www.kaggle.com/nidhirastogi/intrusion-detection/data


In [0]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np
import re
import sklearn

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib as matplot
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
import os

os.environ['KAGGLE_USERNAME'] = "alik604" # username from the json file os.environ['KAGGLEKEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx" # key from the json file
os.environ['KAGGLE_KEY'] = "1c6c5de0f6a4d1610a2f8f5ad6771bb7"

!mkdir /root/.kaggle
!echo '{"username":"alik604","key":"1c6c5de0f6a4d1610a2f8f5ad6771bb7"}' > /root/.kaggle/kaggle.json

!kaggle datasets download -d what0919/intrusion-detection
!ls

mkdir: cannot create directory ‘/root/.kaggle’: File exists
intrusion-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
 intrusion-detection.zip
 mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv
 mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv.1
'mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv?datasetVersionNumber=1'
 sample_data


In [13]:
import zipfile

zip_ref = zipfile.ZipFile('intrusion-detection.zip', 'r')
zip_ref.extractall()
zip_ref.close()
!ls

 intrusion-detection.zip
 mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv
 mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv.1
'mGzrk218cLR0y6St22kr%2Fversions%2FQxtER5hx4vlq4KtjFZ0w%2Ffiles%2Ftest_data.csv?datasetVersionNumber=1'
 sample_data
 test_data.csv
 Train_data.csv


In [0]:
train = pd.read_csv('./Train_data.csv')
test = pd.read_csv('./test_data.csv')
test = test.drop('Unnamed: 0', axis=1)

In [15]:
train.head(5)
df = train.describe()
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,xAttack
0,0,icmp,20,2,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,45,2,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,icmp,50,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,dos
3,0,icmp,25,2,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,icmp,25,2,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


,duration,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,125973.00000,125973.000000,125973.000000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.0,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,32.086288,2.575179,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,0.001342,0.001103,0.302192,0.012669,0.000413,0.004096,0.0,0.000008,0.009423,84.107555,27.737888,0.284485,0.282485,0.119958,0.121183,0.660928,0.063053,0.097322,182.148945,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240
std,2604.51531,16.462574,1.141552,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,0.036603,0.045154,24.399618,0.483935,0.022181,0.099370,0.0,0.002817,0.096612,114.508607,72.635840,0.446456,0.447022,0.320436,0.323647,0.439623,0.180314,0.259830,99.206213,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459
min,0.00000,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,20.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,82.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,25.000000,2.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,14.000000,8.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,255.000000,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,50.000000,4.000000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,143.000000,18.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.060000,0.000000,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42908.00000,70.000000,11.000000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,1.000000,2.000000,7468.000000,43.000000,2.000000,9.000000,0.0,1.000000,1.000000,511.000000,511.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### The following few cells are taken from the 'sample code'

In [0]:
X_train = train.drop('xAttack', axis=1)
Y_train = train.loc[:,['xAttack']]
X_test = test.drop('xAttack', axis=1)
Y_test = test.loc[:,['xAttack']]

In [0]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# enc = preprocessing.OneHotEncoder()
# lb = preprocessing.LabelBinarizer()

In [18]:
X_train['protocol_type'] = le.fit_transform(X_train['protocol_type'])
# enc.fit_transform(X_train['protocol_type'])

X_test['protocol_type'] = le.fit_transform(X_test['protocol_type'])
# enc.fit_transform(X_test['protocol_type'])

X_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0,20,2,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,2,45,2,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,0,50,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,0,25,2,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,0,25,2,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [19]:
set(list(Y_train['xAttack']))

{'dos', 'normal', 'probe', 'r2l', 'u2r'}

In [20]:
Y_train['xAttack'] = le.fit_transform(Y_train['xAttack'])
Y_test['xAttack'] = le.fit_transform(Y_test['xAttack'])
Y_train.describe()

,xAttack
count,125973.000000
mean,0.744985
std,0.653748
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,4.000000


# Drop features with lowest STD

In [0]:
# df.ix[2,:]==0 # see where STD== 0, which means all values are the same... which is useless 
# train = train.drop('num_outbound_cmds', axis=1)

In [22]:
len(train.columns)
train.columns

42

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'xAttack'],
      dtype='object')

In [23]:
## bad smaple code I fixed 
con_list = [
    'protocol_type', 'service', 'flag', 'land', 'logged_in', 'su_attempted',
    'is_host_login', 'is_guest_login'
]

df = X_train.drop(con_list, axis=1)

#drop n smallest std features
df = df.std(axis=0)
df = df.to_frame()

tmp = df.nsmallest(10, columns=0)
tmp = list(
    tmp.transpose().columns)  #fuckin pandas.core.indexes.base.Index   -_-
#tmp = tmp.append('num_outbound_cmds')  # might not work...
tmp

['num_outbound_cmds',
 'urgent',
 'num_shells',
 'root_shell',
 'num_failed_logins',
 'num_access_files',
 'dst_host_srv_diff_host_rate',
 'diff_srv_rate',
 'dst_host_diff_srv_rate',
 'wrong_fragment']

In [0]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import DecisionTreeClassifier




In [25]:
LR = linear_model.LinearRegression()
LR.fit(X_train, Y_train)
lr_score = LR.score(X_test, Y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing ,,,
Linear regression Score: 0.33 %


In [26]:
try:
    X_train = X_train.drop(tmp,axis=1)
    X_test = X_test.drop(tmp,axis=1)
except:
    None
    
X_train.shape
X_test.shape

(125973, 31)

(10000, 31)

In [27]:
LR = linear_model.LinearRegression()
LR.fit(X_train, Y_train)
lr_score = LR.score(X_test, Y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Linear regression processing ,,,
Linear regression Score: 0.21 %


In [0]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, learning_rate=1.0)
RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_features='auto', bootstrap=True)
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')



In [0]:
y_train = Y_train['xAttack'].ravel()
x_train = X_train.values
x_test = X_test.values

In [30]:
AB.fit(X_train, Y_train)


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                          

In [31]:
AB_feature = AB.feature_importances_
AB_feature

ab_score = AB.score(X_test, Y_test)

print('AdaBoostClassifier processing ,,,')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

array([2.39853249e-03,            nan,            nan, 3.96056465e-03,
                  nan, 1.44607765e-02, 1.18537471e-05, 6.82487785e-03,
                  nan, 2.08390004e-04, 2.27099015e-07, 5.27912750e-06,
       9.22977203e-04, 0.00000000e+00, 4.11330432e-04,            nan,
       6.12162178e-03, 3.50789730e-04, 1.75414910e-03, 4.51832499e-03,
       1.65554053e-03, 8.06689778e-03,            nan,            nan,
                  nan, 2.99434364e-01,            nan,            nan,
       1.98833253e-03, 7.92920753e-03, 1.79393433e-04])

AdaBoostClassifier processing ,,,
AdaBoostClassifier Score: 0.755 %


In [32]:
RF.fit(X_train, Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
RF_feature = RF.feature_importances_
RF_feature

rf_score = RF.score(X_test, Y_test)

print('RandomForestClassifier processing ,,,')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

array([4.12527731e-03, 1.67750048e-02, 2.97232607e-02, 5.74194683e-02,
       2.36255213e-01, 1.07979438e-01, 4.84083524e-05, 9.31782519e-03,
       2.06958029e-02, 2.21439857e-03, 2.70854604e-05, 4.67955930e-04,
       2.47755669e-04, 0.00000000e+00, 2.41333372e-03, 8.05441630e-02,
       3.81233559e-02, 4.61376253e-02, 4.08521512e-02, 6.85864806e-03,
       6.58813206e-03, 2.12383725e-02, 1.64956643e-02, 1.97414836e-02,
       4.86244405e-02, 2.45608237e-02, 4.53745625e-02, 1.25960283e-02,
       5.29893980e-02, 3.92915501e-02, 1.22733738e-02])

RandomForestClassifier processing ,,,
RandomForestClassifier Score: 0.753 %


In [34]:
ET.fit(X_train, Y_train)
ET_feature = ET.feature_importances_
ET_feature

et_score = ET.score(X_test, Y_test)

print('ExtraTreesClassifier processing ,,,')
print('ExtraTreeClassifier: %.3f %%' % et_score)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

array([2.39770557e-03, 3.26839303e-02, 2.19238556e-02, 7.47834218e-03,
       2.92996408e-02, 1.07820744e-02, 1.24063767e-04, 8.16523294e-03,
       4.08094374e-02, 4.89547864e-03, 2.42222037e-05, 4.14806617e-04,
       2.52785549e-04, 0.00000000e+00, 2.63057818e-03, 4.57508826e-02,
       1.65792449e-02, 9.18923769e-02, 9.02027072e-02, 1.88471436e-02,
       2.97169399e-02, 8.50771361e-02, 1.56916527e-02, 2.72458348e-02,
       6.32330209e-02, 3.26077010e-02, 5.15946971e-02, 5.92376236e-02,
       1.53039965e-01, 4.63046419e-02, 1.10962781e-02])

ExtraTreesClassifier processing ,,,
ExtraTreeClassifier: 0.767 %


In [0]:
GB.fit(X_train, Y_train)

GB_feature = GB.feature_importances_
GB_feature

gb_score = GB.score(X_test, Y_test)

print('GradientBoostingClassifier processing ,,,')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

In [0]:
cols = X_train.columns.values

feature_df = pd.DataFrame({'features': cols, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
feature_df.features
feature_df.head(8)

In [0]:
a_f = feature_df.nlargest(12, 'AdaBoost')
e_f = feature_df.nlargest(12, 'ExtraTree')
g_f = feature_df.nlargest(12, 'GradientBoost')
r_f = feature_df.nlargest(12, 'RandomForest')

result = pd.concat([a_f, e_f, g_f, r_f])
result = result.drop_duplicates() # duplicate feature삭제
result.shape
result

In [0]:
selected_features = result['features'].values.tolist()
X_train_SF = X_train[selected_features]
X_test_SF = X_test[selected_features]
X_train_SF.shape
X_test_SF.shape

In [0]:
from sklearn.svm import SVC

AB = SVC()#= AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, learning_rate=1.0)
RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_features='auto', bootstrap=True)
ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

AB.fit(x,y)



In [0]:
RF.fit(x,y)

In [0]:
ET.fit(x,y)

In [0]:
GB.fit(x,y)

In [0]:
clf = VotingClassifier([AB,RF,ET,GB],voting='soft') # https://stats.stackexchange.com/a/349556

In [0]:
x = X_train_SF.values#.reshape(-1, 26)  # 31
y = y_train

x.shape
y.size
clf.fit(x,y)


In [0]:
pered =clf.predict(X_test_SF)

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pered,y_test)

In [0]:


AB.fit(x,y)

In [0]:
AB_feature = AB.feature_importances_
AB_feature

ab_score = AB.score(X_test_SF.values, Y_test.values)

print('AdaBoostClassifier processing ,,,')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

In [0]:
len(Y_test.values)

In [0]:
len(X_test_SF.values)